In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
# dataset
from tensorflow.keras.datasets import imdb

# utility and algebraic computations
import numpy as np

# visualisations
import plotly.graph_objects as go

In [ ]:
# num_words parametresi en çok karşılaşılan ilk kaç kelimenin verisetinde bulunacağını belirler
# en çok karşılaşılan 12500 kelimeyi içerecek şekilde veriset döndürelim.
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 12500) 

In [ ]:
# reviewları ingilizceye çevirecek yardımcı bir fonksiyon yazalım
# 0,1,2 indisleri "padding", "start of sequence" ve "unknown" için ayrıldığından, ilk üç indisi kullanamayız
# bu durumda karşılığı olmayan kelimeler görüldüğü taktirde "?" ile yerini değiştireceğiz.
def decode_words(encoded_data):
  word_index = imdb.get_word_index()
  reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])
  decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_data])
  return decoded_review

Eğer veriyi direkt şu anda olduğu gibi gönderirsem model verisetinde kelimelerin aldığı değerler arasında matematiksel bir kural arayabilir, bu yüzden veriyi düzenlemek için popüler 2 farklı yoldan birini kullanacağım.

 1) elimdeki her bir listeyi tensöre dönüştürerek networkte (sample,word_indices) şeklinde tensör alabilecek bir layer kullanabilirim (Embedding gibi)

 2) her birini one-hot-encodelayarak vektörize ederim, böylece elimde
    **max(kelime_değeri)** boyutlu bir vektör olur. Ben 20k kelime aldığımdan
    20k boyutlu bir vektör olacak. Doğal olarak bu yöntem her bir sample için 
    20k boyutunda 1 ve 0 ile kodlanmış vektör oluşturacağından ram'e çok yüklenebilir, duruma bağlı olarak 20k dan 10k kelimeye düşürebilirim
    
`güncelleme: 20k dan 12.5k ya düşürdüm çünkü 5gb ram kullanıyordu vektörizasyon`

In [ ]:
# veriyi vektörize etmek için helper
def vectorize_seqs(sequences, dimension = 12500):
  results = np.zeros((len(sequences), dimension))
  for i, seq in enumerate(sequences):
    results[i, seq] = 1.
  return results

x_train = vectorize_seqs(train_data)
y_train = train_labels
x_test = vectorize_seqs(test_data)
y_test = test_labels

In [ ]:
# modeli eğitirken görmediği veri üzerinde de skor hesaplayabilmek adına antrenman için kullanacağım veriyi 
# parçalayarak doğrulama seti oluşturuyorum. Böylece test verisini kullanmadan da modelin görmediği veri üzerindeki başarısını test edebiliriz.
# aynı zamanda sistemin sabitlerini belirliyorum.
N = 10000
BATCH_SIZE = 512
EPOCHS = 4
x_val = x_train[:N]
y_val = y_train[:N]
partial_x_train = x_train[N:]
partial_y_train = y_train[N:]

In [ ]:
# modelin tanımlandığı blokta modeli compile etmek (derlemek) mantıklı ve doğru bir yaklaşım.
# genelde bir proje üzerinde cross validation uygulanır ve bu yüzden cross validation'a uygun bir proje yapısı oluşturmak gerekir
# bununla ilgili farklı bir örnek daha yapacağım. 
model = Sequential([
  layers.Dense(16, activation = 'relu', input_shape = (12500,)),
  layers.Dense(16, activation = 'relu'),
  layers.Dense(1, activation = 'sigmoid')
])
model.compile(optimizer = RMSprop(lr=0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
h = model.fit(partial_x_train, partial_y_train,
              validation_data = (x_val, y_val),
              batch_size = BATCH_SIZE,
              epochs = EPOCHS
              )

Epoch 1/4
30/30 [==============================] - 2s 53ms/step - loss: 0.5257 - accuracy: 0.7783 - val_loss: 0.4043 - val_accuracy: 0.8754
Epoch 2/4
30/30 [==============================] - 1s 48ms/step - loss: 0.3224 - accuracy: 0.9034 - val_loss: 0.3235 - val_accuracy: 0.8819
Epoch 3/4
30/30 [==============================] - 1s 47ms/step - loss: 0.2304 - accuracy: 0.9321 - val_loss: 0.2813 - val_accuracy: 0.8934
Epoch 4/4
30/30 [==============================] - 1s 47ms/step - loss: 0.1777 - accuracy: 0.9456 - val_loss: 0.2720 - val_accuracy: 0.8948


In [ ]:
# Modelin test verisi üzerindeki performansına bakalım
print("Test verisine modeli uygulayalım")
model.evaluate(x_test, y_test)

Test verisine modeli uygulayalım
782/782 [==============================] - 1s 2ms/step - loss: 0.2940 - accuracy: 0.8832


[0.2939527630805969, 0.8832399845123291]

In [ ]:
# Model üzerinde gradient descent optimizasyonu uyguladığımız zaman Keras bize History objesi döndürüyor.
# Bu obje üzerinde bir history attribute u modelin antrenman esnasında hesapladığı skorları kaydediyor
# Yani bu objeyi kullanarak veri üzerindeki performansını epoch'a göre grafiğe döküp modeli inceleyebiliriz
history_dict = h.history
history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
# Plotly kullanarak antrenman esnasında hesaplanan lossu ve accuracyi grafiğe dökelim
fig = go.Figure()
fig.update_layout(title='Losses',
                  xaxis_title='Epochs',
                  yaxis_title = 'Loss')
fig.add_trace(go.Scatter(x=list(range(EPOCHS)), y = history_dict['loss'], mode = 'lines+markers', name="training loss"))
fig.add_trace(go.Scatter(x=list(range(EPOCHS)), y = history_dict['val_loss'], mode = 'lines+markers', name = 'validation loss'))
fig.show()

fig.data = [] # fig objesinde bulunan yukarıdaki traceler data attribute unda tutulur. 
              # Bunu sıfırlayalım ki aynı objeyi tekrar kullanarak accuracyi grafiğe dökebilelim
fig.update_layout(title='Accuracies',
                  yaxis_title = 'Accuracy')
fig.add_trace(go.Scatter(x=list(range(EPOCHS)), y = history_dict['accuracy'], mode = 'lines+markers', name="training acc"))
fig.add_trace(go.Scatter(x=list(range(EPOCHS)), y = history_dict['val_accuracy'], mode = 'lines+markers', name="validation acc"))
fig.show()

In [ ]:
# Bir kaç örnek gösterebilmek için tahminleri alalım ve doğru-yanlış sınıflandırmaları ayıklayalım
preds = model.predict(x_test) # modeli test verisi üzerinde kullanıp tahminleri alalım
print("Tahminlerden ve gerçek sınıflardan 5'er örnek")
# Sınıflandırmada 1 pozitifi ve 0 negatifi temsil ediyor
print(preds[:5]) # tahminler 0 ile 1 arasında
print(y_test[:5]) # gerçek değerler 0 veya 1
# şimdi tahminler ile test verisinin şekline bakalım
print("Tahminlerin ve gerçek sınıfların şekli")
print(preds.shape) # (25000,1)
print(y_test.shape) # (25000,)
# boyutları da birbirinden farklı olduğu için tahminleri düzleştirelim yani sağda gözüken ekstra boyutu kaldırarak (25000,) olmasını sağlayalım
preds = preds.flatten()
preds = np.round(preds) # tahminler 0 ile 1 arasında olasılıklardan oluştuğu için yuvarlayarak 0 veya 1 olmasını sağlayalım.

print("Dönüşümler yapıldıktan sonra:")
print("Tahminlerin şekli:",preds.shape)
print("Tahminlerden 5 örnek:", preds[:5])
print("Gerçek sınıfların şekli:",y_test.shape)
print("Gerçek sınıflardan 5 örnek:",y_test[:5])

Tahminlerden ve gerçek sınıflardan 5'er örnek
[[0.25026232]
 [0.99267006]
 [0.8712561 ]
 [0.6239792 ]
 [0.84120405]]
[0 1 1 0 1]
Tahminlerin ve gerçek sınıfların şekli
(25000, 1)
(25000,)
Dönüşümler yapıldıktan sonra:
Tahminlerin şekli: (25000,)
Tahminlerden 5 örnek: [0. 1. 1. 1. 1.]
Gerçek sınıfların şekli: (25000,)
Gerçek sınıflardan 5 örnek: [0 1 1 0 1]


In [ ]:
# yazının sağa doğru taşmaması için pretty print ile bir genişlik belirleyelim
import pprint
pp = pprint.PrettyPrinter(width = 80, compact = True)

# artık boyutları da yapıları da aynı olduğundan doğru olan/olmayan sınıflandırmaları ayıklayabiliriz
misclassified_idx = np.where(preds != y_test)[0]
correct_idx = np.where(preds == y_test)[0]

# yanlış sınıflandırılanlardan rastgele bir örnek
r = np.random.choice(misclassified_idx)
review = test_data[r]
review = decode_words(review)
print("Yanlış sınıflandırma örneği\n\n")
print(f"Tahmin: {preds[r]}, Gerçek: {test_labels[r]}")
pp.pprint(review)
# doğru sınıflandırmalardan rastgele bir örnek
r = np.random.choice(correct_idx)
review = test_data[r]
review = decode_words(review)
print("\n\nDoğru sınıflandırma örneği\n\n")
print(f"Sınıf: {preds[r]}")
pp.pprint(review)

Yanlış sınıflandırma örneği


Tahmin: 1.0, Gerçek: 0
("? am not from america i usually watch this show on channel i don't know why "
 'this respected channel air such sucking program in prime time slot creation '
 "of hollywood's money bank jerry bruckheimer this time he is spending a big "
 'load of cash in the small screen in each episode a bunch of peoples having '
 'two team members travels from on country to another for a great sum of money '
 "where the camera ? shoot their travels i don't know who the hell gave this "
 'stupid idea for the show it has nothing to watch for in all episodes we see '
 "people ran like ? some times shouting crying ? jerky camera works huh it's ? "
 'to both eyes and ears the most disgusting part in the race is the viewers '
 "finally knows each of the team members can't enjoy their race traveling "
 'experience even though to add up the ratings the producers came up with the '
 'ideas of including gays in one shows sucking american reality show br br